# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75.

## 1. Откройте и изучите файл

In [56]:
#Импортируем необходимые библиотеки
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import randint
import warnings 
warnings.filterwarnings("ignore") 

In [8]:
#откроем файл с данными 
df = pd.read_csv('/home/andrey/Datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Необходимые библиотеки загружены, файл открыт. Данные уже предобработанны.

## 2. Обучите модели

In [14]:
#разобьём данные на train и test

y = df['is_ultra']
X = df.drop('is_ultra', axis = 1)


from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25, random_state = 12345)

Используем следующие модели для обучения:

- Decision Tree
- Random Forest
- Logistic Regression
- KNN

In [42]:
%%time
#decision tree
model = DecisionTreeClassifier(random_state = 12345)

#подберём гиперпараметры с помощью RandomizedSearchCV
param = {"max_depth": [3, 4, 5, 6, 7, 8, 9], 
         "max_features": randint(1, 9),
         "min_samples_leaf": randint(1, 9),
         "criterion": ["gini", "entropy"],
        "splitter" : ["best", "random"],
        "class_weight" : ["balanced", None]}

model_rscv = RandomizedSearchCV(model, param_distributions = param, scoring = "accuracy",
                             cv = 2, verbose = 0, random_state = 12345, n_jobs = -1, n_iter = 1000)

model_tree = model_rscv.fit(X_train, y_train)

print("Лучшие гиперпараметры для модели дерева решений")
print()
print(model_tree.best_estimator_.get_params())

pred = model_tree.predict(X_test)
print()
print("Accuracy = {}".format(accuracy_score(y_test, pred)))

Лучшие гиперпараметры для модели дерева решений

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 7, 'max_features': 3, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 7, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': 'deprecated', 'random_state': 12345, 'splitter': 'best'}

Accuracy = 0.7810945273631841
CPU times: user 2.15 s, sys: 32 ms, total: 2.19 s
Wall time: 2.39 s


С помощью подбора гиперпараметров удалось достичь значения accuracy 0.7898

In [43]:
%%time
#random_forest
model = RandomForestClassifier(random_state = 12345)

#подберём гиперпараметры с помощью RandomizedSearchCV
param = {"max_depth": [3, 4, 5, 6, 7, 8, 9, 11, 13, 15, 17, 19, 21],
         "n_estimators" : randint(500, 4000),
         "min_samples_split" : randint(1,10),
         "min_samples_leaf": randint(1, 9),
         "criterion": ["gini", "entropy"],
        "class_weight" : ["balanced", None]}

model_rscv = RandomizedSearchCV(model, param_distributions = param, scoring = "accuracy",
                             cv = 2, verbose = 0, random_state = 12345, n_jobs = -1, n_iter = 200)

model_forest = model_rscv.fit(X_train, y_train)

print("Лучшие гиперпараметры для модели случайного леса")
print()
print(model_forest.best_estimator_.get_params())

pred = model_forest.predict(X_test)
print()
print("Accuracy = {}".format(accuracy_score(y_test, pred)))

Лучшие гиперпараметры для модели случайного леса

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 11, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 3, 'min_samples_split': 3, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 2734, 'n_jobs': None, 'oob_score': False, 'random_state': 12345, 'verbose': 0, 'warm_start': False}

Accuracy = 0.804726368159204
CPU times: user 8.68 s, sys: 47.4 ms, total: 8.73 s
Wall time: 2min 54s


В модели слуайного леса удалось увеличить значение accuracy до 0.8047

In [47]:
%%time
#logistic regression
model = LogisticRegression(random_state=12345)

param = {"class_weight" : ['balanced', None],
         "penalty" : ["l1", "l2", "elasticnet", "None"],
         "solver" : ["liblinear", "lbfgs"]
        }

model_rscv = RandomizedSearchCV(model, param_distributions = param, scoring = "accuracy",
                             cv = 2, verbose = 0, random_state = 12345, n_jobs = -1, n_iter = 100)

model_lr = model_rscv.fit(X_train, y_train)

print("Лучшие гиперпараметры для модели логистической регрессии")
print()
print(model_lr.best_estimator_.get_params())

pred = model_lr.predict(X_test)
print()
print("Accuracy = {}".format(accuracy_score(y_test, pred)))

Лучшие гиперпараметры для модели логистической регрессии

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 12345, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

Accuracy = 0.7039800995024875
CPU times: user 82.1 ms, sys: 0 ns, total: 82.1 ms
Wall time: 86.8 ms


Модель логистической регрессии показала самое низкое качество по метрике accuracy 0.7039

In [55]:
%%time
#KNN
model = KNeighborsClassifier()

param = {"n_neighbors" : randint(1 , 10),
         "weights" : ["uniform", "distance"],
         "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"]}
         
model_KNN = RandomizedSearchCV(model, param_distributions = param, scoring = "accuracy",
                             cv = 2, verbose = 0, random_state = 12345, n_jobs = -1, n_iter = 100)

model_KNN = model_KNN.fit(X_train, y_train)

print("Лучшие гиперпараметры для KNN")
print()
print(model_KNN.best_estimator_.get_params())

pred = model_KNN.predict(X_test)
print()
print("Accuracy = {}".format(accuracy_score(y_test, pred)))

Лучшие гиперпараметры для KNN

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 9, 'p': 2, 'weights': 'distance'}

Accuracy = 0.7736318407960199
CPU times: user 391 ms, sys: 57.2 ms, total: 449 ms
Wall time: 1.23 s


Accuracy в KNN составил 0.7736

Лучший accuracy с тюнингом гиперпараметров показала модель случайного леса.

## 3. Проверьте модели на адекватность

In [53]:
#проверим модель на адекватность с помощью DummyClassifier
dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(X_train, y_train)
predict_dummy = dummy_clf.predict(y_test)
accuracy_dummy = accuracy_score(y_test, predict_dummy)
print('accuracy на cлучайной выборке:' ,accuracy_dummy)



accuracy на cлучайной выборке: 0.49129353233830847


С помощью DummyClassifier модель random forest была проверена на адекватность, результат DummyClassifier оказался ниже чем у всех примененных моделях.

## 4. Вывод

В ходе работы было построено несколько различных рекомендательных моделей машинного обучения с перебором гиперпараметров.

Было применено 4 модели:
- Decision Tree
- Random Forest
- Logistic Regression
- KNN

Лучший результат показала модель случайного леса с точностью 80%, следом за ней идёт модель дерева решений с точностью 78%. 

Так же стоит отметить, что при не большой разности в качестве метрик у обоих моделей, модель дерева решений работает намного быстрее случайного леса. Поэтому для скорости стоит применять именного дерево решений. а для большей точности случайный лес.